In [27]:
import torch
import torch.utils.data as Data 
from torch.nn import init
import torch.optim as optim 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from collections import OrderedDict
import numpy as np
import time 
import sys
from IPython import display
from matplotlib import pyplot as plt

In [2]:
# define a fliter from cross correlation
def corr2d(X, K):
    l, w = K.shape[0], K.shape[1]
    Y = torch.zeros(X.shape[0]-l+1, X.shape[1]-w+1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+l, j:j+w]*K).sum()
    return Y   


In [3]:
#define a fliter from convolutional view
def corr2d_c(X, K):
    l, w = K.shape[0], K.shape[1]
    Y = torch.zeros(X.shape[0]-l+1, X.shape[1]-w+1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            summation = 0
            for m in range(K.shape[0]):
                for n in range(K.shape[1]):
                    summation += X[i+m,j+n]*K[m,n]
            Y[i,j] = summation
    return Y    
# I feel very  with this function
# I fe function


In [5]:
X = torch.tensor([[0,1,2],[3,4,5],[6,7,8]])
K = torch.tensor([[0,1],[2,3]])
print(corr2d(X, K))
print(corr2d_c(X,K))

tensor([[19., 25.],
        [37., 43.]])
tensor([[19., 25.],
        [37., 43.]])


In [6]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size)) 
        self.bias = nn.Parameter(torch.randn(1))
    def forward(self, X):
        return self.corr2d(X, self.weight)+self.bias      
    @staticmethod
    def corr2d(X, K):
        l, w = K.shape[0], K.shape[1]
        Y = torch.zeros(X.shape[0]-l+1, X.shape[1]-w+1)
        for i in range(Y.shape[0]):
            for j in range(Y.shape[1]):
                Y[i,j] = (X[i:i+l, j:j+w]*K).sum()
        return Y  
    
#@classmethod隐含class的self被当作参数
#

In [7]:
X = torch.ones(8,8, dtype = torch.float)
X[:, 2:6] = 0.0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [8]:
K = torch.tensor([[1,-1]])
Y = Conv2D.corr2d(X, K)
print(Y)
#it can represent the feature of space very well

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])


In [9]:
conv2d = Conv2D((1,2))

In [10]:
# through the supervision learning to learn the parameter of the kernel
# the standard kernel K
K = torch.tensor([[1,-1]])
Y = Conv2D.corr2d(X, K) # recall the staticmethod

num_epoch = 30 
lr = 0.01      
conv2d = Conv2D((1,2))     

#upgrade the kernel in conv2d
for epoch in range(num_epoch):
    Y_hat = conv2d(X)
    loss = ((Y - Y_hat)**2).sum()
    
    if conv2d.weight.grad is not None:
        for _, param in conv2d.named_parameters():
            param.grad.zero_()
    
    loss.backward()
    
    for param in conv2d.parameters():
        param.data -= lr*param.grad
    
    if (epoch+1)%5 == 0:
        print('epoch:{0}, loss:{1}'.format(epoch, loss))
print(conv2d.weight.data, conv2d.bias.data)

epoch:4, loss:7.197460174560547
epoch:9, loss:0.9880686402320862
epoch:14, loss:0.16810810565948486
epoch:19, loss:0.02930174581706524
epoch:24, loss:0.005121978931128979
epoch:29, loss:0.0008957354002632201
tensor([[ 0.9937, -0.9937]]) tensor([2.2205e-05])


In [14]:
list(conv2d.named_parameters())

[('weight',
  Parameter containing:
  tensor([[ 0.4457, -0.0815]], requires_grad=True)),
 ('bias',
  Parameter containing:
  tensor([1.2588], requires_grad=True))]

In [40]:
# Padding and Stride
X = torch.rand(8,8)

In [44]:
conv2d = nn.Conv2d(in_channels = 1, out_channels = 1, kernel_size = 3, padding = 1)

In [45]:
def comp_conv2d(conv2d_c, x_c):
    x_c = x_c.view((1,1)+x_c.shape)
    y = conv2d_c(x_c)
    return y.view(y.shape[2:])

In [48]:
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [50]:
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size = (5,3), padding=(2,1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [51]:
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size = (3,5), padding=(0,1), stride=(3, 4))
comp_conv2d(conv2d,X)   

tensor([[-0.2330, -0.6291],
        [-0.3928, -0.5024]], grad_fn=<ViewBackward>)

In [52]:
# define a fliter from cross correlation
def corr2d(X, K):
    l, w = K.shape[0], K.shape[1]
    Y = torch.zeros(X.shape[0]-l+1, X.shape[1]-w+1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+l, j:j+w]*K).sum()
    return Y    

In [53]:
def corr2d_multi_in(X,K):
    res = corr2d(X[0,:,:], K[0,:,:])
    for i in range(1, X.shape[0]):
        res += corr2d(X[i,:,:], K[i,:,:])
    return res

In [66]:
X = torch.tensor([[[0,1,2],[3,4,5],[6,7,8]],[[1,2,3],[4,5,6],[7,8,9]]])
K = torch.tensor([[[1,2],[3,4]],[[5,6],[7,8]]])

In [60]:
corr2d_multi_in(X,K).shape

torch.Size([2, 2])

In [62]:
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K])

In [67]:
K = torch.stack([K,K+1,K+2])
for k in K:
    print(k.shape)

torch.Size([2, 2, 2])
torch.Size([2, 2, 2])
torch.Size([2, 2, 2])


In [69]:
corr2d_multi_in_out(X, K).shape

torch.Size([3, 2, 2])

In [72]:
def corr2d_multi_in_out_1_1(X, K):
    c,h,w = X.shape
    X = X.view(c,-1)
    K = K.view(K.shape[0],-1)
    Y = torch.mm(K,X)
    return Y.view(K.shape[0], h, w)

In [73]:
X = torch.rand(3,3,3)
K = torch.rand(4,3,1,1)

In [75]:
corr2d_multi_in_out_1_1(X,K).shape

torch.Size([4, 3, 3])

In [78]:
corr2d_multi_in_out(X,K) == corr2d_multi_in_out_1_1(X,K)

tensor([[[ True,  True,  True],
         [ True,  True,  True],
         [ True, False,  True]],

        [[ True,  True,  True],
         [ True,  True, False],
         [False,  True,  True]],

        [[ True,  True,  True],
         [ True, False, False],
         [ True,  True, False]],

        [[ True,  True,  True],
         [ True,  True,  True],
         [ True,  True,  True]]])

In [13]:
X = torch.arange(16, dtype = torch.float).view((1,1,4,4))
pool2d = nn.MaxPool2d(3) #default set the same as the stride
pool2d(X)
pool2d=nn.MaxPool2d(2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [8]:
pool2d = nn.MaxPool2d(kernel_size=(2,4),padding=(1,2),stride=(2,3))
pool2d(X)

tensor([[[[ 1.,  3.],
          [ 9., 11.],
          [13., 15.]]]])

In [15]:
X = torch.cat((X, X+1),dim=1)


In [16]:
pool2d = nn.MaxPool2d(kernel_size=3,padding=1,stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])

In [18]:
class Flatten(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,x):
        return x.view(x.shape[0],-1)

In [21]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5),
                                  nn.Sigmoid(),
                                  nn.MaxPool2d(kernel_size=2,stride=2),
                                  nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5),
                                  nn.Sigmoid(),
                                  nn.MaxPool2d(kernel_size=2,stride=2)
                                 )
        self.flatten = nn.Sequential(Flatten())
        self.fc = nn.Sequential(nn.Linear(16*4*4, 120),
                               nn.Sigmoid(),
                               nn.Linear(120,84),
                               nn.Sigmoid(),
                               nn.Linear(84,10))
    def forward(self,X):
        conv_feature = self.conv(X)
        flat_feature = self.flatten(conv_feature)
        output = self.fc(flat_feature)
        return output 
    

In [25]:
def load_data_fashion_mnist(batch_size):
    mnist_train = torchvision.datasets.FashionMNIST(root = './Dataset/FashionMNIST', train = True, 
                                                    download = True, transform = transforms.ToTensor())
    mnist_test = torchvision.datasets.FashionMNIST(root = './Dataset/FashionMNIST', train = False, 
                                                   download = True, transform = transforms.ToTensor())
    if sys.platform.startswith('win'):
        num_workers = 4
    else:
        num_workers = 0
    train_iter = Data.DataLoader(mnist_train, batch_size = batch_size, shuffle = True, num_workers = num_workers)
    test_iter = Data.DataLoader(mnist_test, batch_size = batch_size, shuffle = True, num_workers = num_workers)
    return train_iter, test_iter

In [29]:
batch_size = 256
train_iter, test_iter = load_data_fashion_mnist(batch_size)

In [59]:
def evaluate_accuracy(data_iter, network, device = None):
    acc_num, n = 0.0, 0
    if device is None:        
        device = list(network.parameters())[0].device
    with torch.no_grad():
        for X, y in data_iter: 
            network.eval()
            num = ((network(X.to(device))).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
            acc_num += num 
            network.train()
            n += y.shape[0]
    return acc_num/n

In [60]:
def training_loop(network, train_iter, test_iter, optimizer, num_epochs): 
    device = 'cuda' if torch.cuda.is_available else 'cpu' 
    network = network.to(device)
    print('training on ', device)
    loss_fn = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        train_loss_sum, train_acc_num, test_acc, batch_count, n, start_time = 0.0, 0.0, 0.0, 0, 0, time.time()
        for X, y in train_iter:
            X = X.to(device)
            y = y.to(device)
            y_hat = network(X)
            loss = loss_fn(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss_sum += loss.cpu().item() 
            train_acc_num += (y_hat.argmax(dim=1)== y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        train_acc = train_acc_num/n
        train_loss = train_loss_sum/batch_count
        test_acc += evaluate_accuracy(test_iter, network)
        for X_test, y_test in test_iter:
            test_loss_sum, batch_count_test = 0.0, 0
            X_test = X_test.to(device)
            y_test = y_test.to(device)
            with torch.no_grad():
                y_test_hat = network(X_test)
                test_loss_sum += loss_fn(y_test_hat, y_test).cpu().item() 
            batch_count_test += 1
        test_loss = test_loss_sum/batch_count_test
        print('epoch: %d, time: %.3f, train_loss: %.3f, test_loss: %.3f, train_acc: %.2f, test_acc: %.2f'
              %(epoch+1, (time.time()-start_time), train_loss, test_loss, train_acc, test_acc))

In [62]:
network = LeNet()
print(network)

LeNet(
  (conv): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Sequential(
    (0): Flatten()
  )
  (fc): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [63]:
lr, num_epochs = 0.001, 5
optimizer = optim.Adam(network.parameters(), lr=lr)
training_loop(network, train_iter, test_iter, optimizer, num_epochs=5)


training on  cuda:0
epoch: 1, time: 4.166, train_loss: 1.834, test_loss: 0.852, train_acc: 0.32, test_acc: 0.60
epoch: 2, time: 4.107, train_loss: 0.941, test_loss: 0.550, train_acc: 0.64, test_acc: 0.68
epoch: 3, time: 4.080, train_loss: 0.774, test_loss: 0.842, train_acc: 0.72, test_acc: 0.72
epoch: 4, time: 4.058, train_loss: 0.685, test_loss: 0.863, train_acc: 0.74, test_acc: 0.74
epoch: 5, time: 3.988, train_loss: 0.630, test_loss: 0.309, train_acc: 0.76, test_acc: 0.75
